In [103]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [104]:
import yaml

In [105]:
from IPython.html.widgets import FloatProgress
from IPython.display import display

In [106]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [107]:
saut="\n"

### Préparation des matrices de traits

In [108]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [109]:
phonologicalMap="-N"
sampleNumber="01%s"%phonologicalMap
sampleType="BRASS"
#sampleType="SILVER"
samplePrefix="MGC-160104-%s-ext3-derivations-%s"%(sampleNumber,sampleType)
#samplePrefix="MGC-160104-%s-ext3-160215-derivations-%s"%(sampleNumber,sampleType)
sampleFile=samplePrefix+".csv"
analysisPrefix=samplePrefix

In [110]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [111]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [112]:
#recoder("E96Oo")

In [113]:
if not 'formesScoresSamples' in globals():
    formesScoresSamples={}

In [114]:
paradigmes=pd.read_csv("2015-Data/"+sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [115]:
basesDerives=pd.read_csv("2015-Data/MGC-160104-01-N-basesDerives.csv",sep=";",encoding="utf8")

In [116]:
with open("2015-Data/"+analysisPrefix+'-Scores.pkl', 'rb') as input:
    fScores=pickle.load(input)
(formesScores,formesScoresNormes)=fScores

In [117]:
with open("2015-Data/"+analysisPrefix+'-TousDerives.pkl', 'rb') as input:
    tousDerives=pickle.load(input)

In [118]:
trainingDerives=tousDerives[["lexeme","FS_x","FP_x"]].copy()
trainingDerives.rename(columns={"lexeme":'base'},inplace=True)
#trainingDerives["FS_x"]=recoder(trainingDerives["FS_x"])
#trainingDerives

In [119]:
with open("2015-Data/MGC-160104-DerivesComplets.pkl", 'rb') as input:
    derivesComplets=pickle.load(input)
derivesComplets["FS"]=derivesComplets.apply(lambda x: recoder(x["FS"]),axis=1)
derivesComplets["FP"]=derivesComplets.apply(lambda x: recoder(x["FP"]),axis=1)
derivesComplets.loc[derivesComplets["FS"].isnull(),"FS"]=derivesComplets.loc[derivesComplets["FS"].isnull(),"FP"]
derivesComplets.loc[derivesComplets["FP"].isnull(),"FP"]=derivesComplets.loc[derivesComplets["FP"].isnull(),"FS"]

In [120]:
derivesComplets=pd.merge(derivesComplets,trainingDerives,on="base")
derivesComplets

ms,base,FS,FP,FS_x,FP_x
0,absoudre,absut,absut,NaN,NaN
1,accalmir,akalmi,akalmi,NaN,NaN
2,accroître,akry,akry,NaN,NaN
3,agonir,agOni,agOni,NaN,NaN
4,aller,ale,ale,ale,NaN
5,amener,am9ne,am9ne,NaN,NaN
6,arriver,arive,arive,arive,NaN
7,asseoir,asiz,asiz,NaN,NaN
8,astreindre,astrêt,astrêt,NaN,NaN
9,atteindre,atêt,atêt,NaN,NaN


# Le tableau est assemblé à la main à partir des échantillons
- faire une boucle pour charger tous les échantillons

In [121]:
with open("2015-Data/MGC-160104-ScoresComplets.pkl", "rb") as input:
    formesScoresSamples=pickle.load(input)
#formesScoresSamples

In [122]:
numerosSamples=["1","2","3"]
typesSamples=["B","S"]
nomsSamples=[n+t for (n,t) in (list(it.product(numerosSamples, typesSamples)))]
casesSamples=["FS","FP"]
samplesColumnsFormes=[c+"-"+n for (n,c) in (list(it.product(nomsSamples,casesSamples)))]
samplesColumns=["base","FS","FP","FS_x","FP_x"]+samplesColumnsFormes
samplesEmptyColumn=["" for element in samplesColumns]

In [123]:
samplesSelections=derivesComplets.copy()
samplesSelections

ms,base,FS,FP,FS_x,FP_x
0,absoudre,absut,absut,NaN,NaN
1,accalmir,akalmi,akalmi,NaN,NaN
2,accroître,akry,akry,NaN,NaN
3,agonir,agOni,agOni,NaN,NaN
4,aller,ale,ale,ale,NaN
5,amener,am9ne,am9ne,NaN,NaN
6,arriver,arive,arive,arive,NaN
7,asseoir,asiz,asiz,NaN,NaN
8,astreindre,astrêt,astrêt,NaN,NaN
9,atteindre,atêt,atêt,NaN,NaN


In [124]:
#numLexeme=0
for sample in formesScoresSamples:
    sampleName="".join(sample.split("-"))
    if debug or 0: print (sampleName)
    for lexeme in formesScoresSamples[sample]:
        if debug or 0: print (lexeme)
        for cell in formesScoresSamples[sample][lexeme]:
            if debug or 0: print (cell)
            formeScores=formesScoresSamples[sample][lexeme][cell]
            try: 
                forme=max(formeScores, key=lambda i: formeScores[i])
            except ValueError:
                forme=np.NaN
            if debug or 0: print (forme)
#            if not lexeme in samplesSelections["base"].tolist():
#                if debug or 1: print ("adding",lexeme)
#                samplesSelections.loc[:, "base"]=lexeme
#                numLexeme+=1
            samplesSelections.loc[samplesSelections["base"]==lexeme,cell+"-"+sampleName]=forme

In [125]:
def fNoLex(df):
    return ((df["FS_x"].isnull())&(df["FP_x"].isnull()))
def fFS(df,nom):
    return ((df["FS-%s"%nom].notnull())&(df["FP-%s"%nom].isnull()))
def fFP(df,nom):
    return ((df["FP-%s"%nom].notnull())&(df["FS-%s"%nom].isnull()))
def fFSFP(df,nom):
    return ((df["FS-%s"%nom].notnull())&(df["FP-%s"%nom].notnull()))


def fPrecision(df,case,nom):
    return ((df["%s-%s"%(case,nom)].notnull())&(df["%s_x"%(case)].isnull()))

def filtrePrecisionRappel(df,case,nom):
    return ((df[case].notnull())&(df["%s_x"%(case)].isnull())&(df["%s-%s"%(case,nom)].notnull()))

In [126]:
for nom in nomsSamples:
    print (nom, end="\t")
    for case in casesSamples:
        print (case, end="\t")
        filtrePrecision=filtrePrecisionRappel(samplesSelections,case,nom)
        filtreRappel=filtrePrecisionRappel(samplesSelections,case,nom)
        ok=samplesSelections[filtrePrecision].apply(lambda x: x[case]==x["%s-%s"%(case,nom)],axis=1).sum()
        mauvais=samplesSelections[filtrePrecision].apply(lambda x: x[case]!=x["%s-%s"%(case,nom)],axis=1).sum()
        total=samplesSelections[filtreRappel].apply(lambda x: x[case]==x[case],axis=1).sum()
        print (total,"\t", ok,"\t", mauvais,end="\t")
    print ()

1B	FS	137 	 83 	 54	FP	115 	 69 	 46	
1S	FS	130 	 56 	 74	FP	170 	 83 	 87	
2B	FS	137 	 80 	 57	FP	115 	 67 	 48	
2S	FS	130 	 57 	 73	FP	170 	 84 	 86	
3B	FS	137 	 77 	 60	FP	115 	 65 	 50	
3S	FS	130 	 56 	 74	FP	170 	 83 	 87	


In [127]:
#mauvais["base"]

IndexError: invalid index to scalar variable.

In [128]:
for nom in nomsSamples:
    print (nom, end="\t")
    fNomNoLex=fNoLex(samplesSelections)
    fNomFS=fFS(samplesSelections,nom)
    fNomFP=fFP(samplesSelections,nom)
    fNomFSFP=fFSFP(samplesSelections,nom)
    ok=samplesSelections[fNomNoLex&fNomFS].apply(lambda x: x["FS"]==x["FS-%s"%(nom)],axis=1).sum()
    ok+=samplesSelections[fNomNoLex&fNomFP].apply(lambda x: x["FP"]==x["FP-%s"%(nom)],axis=1).sum()
    ok+=samplesSelections[fNomNoLex&fNomFSFP].apply(lambda x: (x["FS"]==x["FS-%s"%(nom)]) and (x["FP"]==x["FP-%s"%(nom)]),axis=1).sum()
    mauvais=samplesSelections[fNomNoLex&fNomFS].apply(lambda x: x["FS"]!=x["FS-%s"%(nom)],axis=1).sum()
    mauvais+=samplesSelections[fNomNoLex&fNomFP].apply(lambda x: x["FP"]!=x["FP-%s"%(nom)],axis=1).sum()
    mauvais+=samplesSelections[fNomNoLex&fNomFSFP].apply(lambda x: (x["FS"]!=x["FS-%s"%(nom)]) or (x["FP"]!=x["FP-%s"%(nom)]),axis=1).sum()
    total=samplesSelections[fNomNoLex].apply(lambda x: x[case]==x[case],axis=1).sum()
    print (total,"\t", ok["base"],"\t", mauvais["base"],end="\t")
    print ()

1B	129 	 73.0 	 56.0	
1S	129 	 51.0 	 72.0	
2B	129 	 73.0 	 56.0	
2S	129 	 52.0 	 71.0	
3B	129 	 71.0 	 58.0	
3S	129 	 51.0 	 72.0	


In [60]:
samplesSelections[filtrePrecisionRappel(samplesSelections,"FP","1S")][["base","FP","FP_x","FP-1S"]]
#formesScoresSamples

ms,base,FP,FP_x,FP-1S
0,absoudre,absut,NaN,absut
1,accalmir,akalmi,NaN,akalmiz
3,agonir,agOni,NaN,agOnit
4,aller,ale,NaN,vat
6,arriver,arive,NaN,arive
7,asseoir,asiz,NaN,aswat
12,battre,baty,NaN,bat
16,buer,bye,NaN,byt
17,buter,byte,NaN,bytet
18,charpir,Sarpi,NaN,Sarpit
